In [85]:
import time

In [86]:
allstart = time.time()

In [87]:
import pandas as pd
import pprint
import re
from nltk.corpus import wordnet as wn
from collections import deque
import re
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

In [88]:
HPO_FILE = 'data/all_hpo_terms_and_synonyms.txt'
PHENOTYPE_FILE = 'data/sample_patient_phenotype.txt'
PHENOTYPE_TO_GENE_FILE = 'data/Expanded_OMIM_ALL_FREQUENCIES_phenotype_to_genes_without_synonym.txt'
PHENOTYPE_TO_DISEASE_FILE = 'data/Expanded_ALL_SOURCES_ALL_FREQUENCIES_diseases_to_phenotypes.txt'

In [89]:
start = time.time()

df_genes = pd.read_csv('data/sample_genes.txt', sep = '\t')
CANDIDATE_GENES = pd.unique(df_genes.Gene.values).tolist()

end = time.time()
print(end - start)

0.0100679397583


In [90]:
OUT_FILE = 'result/ranking_genes.txt'
OUT_FILE_DISEASE = 'result/ranking_diseases.txt'

In [91]:
start = time.time()

hpo_subclass = dict()
with open('data/hpo_subclasses.txt', 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid = parts[0]
        subclasses = parts[1].strip('[]').replace("'", "")
        subclasses = [_.strip() for _ in subclasses.split(',')]
        hpo_subclass[hpoid] = subclasses

hpo_levels = dict()
level = 0
root = 'HP:0000001'
curr_level_nodes = [root] 
while curr_level_nodes:
    next_level_nodes = []
    for node in curr_level_nodes:
        hpo_levels[node] = level
        if node in hpo_subclass:
            next_level_nodes += hpo_subclass[node]
    curr_level_nodes = next_level_nodes
    level += 1
     

hpo_superclass = dict()
with open('data/hpo_superclasses.txt', 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid = parts[0]
        superclasses = parts[1].strip('[]').replace("'", "")
        superclasses = [_.strip() for _ in superclasses.split(',')]
        hpo_superclass[hpoid] = superclasses

hpo_name = dict()
with open('data/hpo_names.txt', 'rb') as f:
    # Skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid = parts[0]
        hponame = parts[1]
        hpo_name[hpoid] = hponame        

end = time.time()
print(end - start)

0.127974033356


In [92]:
tree = hpo_superclass
def getAllAncestorsBFS(node, tree):
    P, Q = [node], deque([node])
    while Q:
        u = Q.popleft()
        if u not in tree:
            continue
        for v in tree[u]:
            if v in P: continue
            if v not in tree: continue
            P.append(v)
            Q.append(v)
    # Append All ('HP:0000001') as the highest ancestor 
    P.append('HP:0000001')
    return P 

def findLowestCommonAncestor(node1, node2, tree):
    node1_ancestors = getAllAncestorsBFS(node1, tree) 
    node2_ancestors = getAllAncestorsBFS(node2, tree)
    common_ancestors = [] 
    for ancestor in node2_ancestors:
        if ancestor in node1_ancestors:
            name = hpo_name[ancestor]
            level = hpo_levels[ancestor]
            common_ancestors.append((ancestor, level, name)) 
    lca = common_ancestors[0]
    return lca, common_ancestors

## Generate antonyms of a word using NLTK and wordnet
def get_antonyms(word):
    antonyms = []
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            antonyms += lemma.antonyms()
            similar_synsets = synset.similar_tos() 
            for similar_synset in similar_synsets:
                for lemma_ in similar_synset.lemmas():
                    #print (similar_synset, lemma_, lemma_.name(), 
                    #      lemma_.antonyms(), similar_synset.similar_tos())
                    antonyms += lemma_.antonyms()
    direct_antonyms = list(set(antonyms))
    # Get similar words of the direct antonyms
    for antonym in direct_antonyms:
        similar_synsets = antonym.synset().similar_tos()
        for synset in similar_synsets:
            for lemma in synset.lemmas():
                antonyms.append(lemma)
    antonyms = list(set(antonyms))
    antonyms = set([_.name() for _ in antonyms])
    return antonyms

In [93]:
start = time.time()

## Map patient phenotypes to hpo standard terms 
corner_cases = dict()
with open(PHENOTYPE_FILE, 'rb') as infile:
    phenos = []
    for line in infile:
        if line.startswith('#') or not line.strip():
            continue
        line = line.rstrip()
        phenos += line.split(',')
        for pheno in phenos:
            if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
                phenos.append('growth delay')
                corner_cases['growth delay'] = pheno.strip()
        for pheno in phenos:
            if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
                phenos.append('developmental delay')
                corner_cases['developmental delay'] = pheno.strip()
    phenos = [_.strip() for _ in phenos]
# print phenos


with open(HPO_FILE, 'rb') as infile:
    hpo_name2id = dict()
    hpo_id2synonyms = dict()
    # skip the first line
    infile.readline()
    for line in infile.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid, hponame = parts[0], parts[1]
        hpo_name2id[hponame] = hpoid
        hpoid = hpoid.split('-')[0]
        if hponame in hpo_id2synonyms:
            hpo_id2synonyms[hpoid].append(hponame)
        else:
            hpo_id2synonyms[hpoid] = [hponame]
    hpo_terms = hpo_name2id.keys()  


end = time.time()
print(end - start)

0.898631811142


In [94]:
start = time.time()

# Use re for split hpo terms
# Use lemmatizer to lemmatize words e.g., disturbances --> distrubance
def isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
    # If any word ends with ness, convert to its noun form
    words_in_pheno_only |= set([_[0:-4] for _ in words_in_pheno_only if _.endswith('ness')])
    words_in_hpo_only |= set([_[0:-4] for _ in words_in_hpo_only if _.endswith('ness')])
    # corner case: gain
    if 'gain' in words_in_hpo_only:
        words_in_hpo_only.add('increased')
        words_in_hpo_only.add('steady')
    if 'gain' in words_in_pheno_only:
        words_in_pheno_only.add('increased')
        words_in_pheno_only.add('steady')
    # Poor weight gain and weight gain are in opposite meaning
    if (words_in_pheno_only == set(['poor']) and words_in_hpo_only == set()) or (words_in_hpo_only == set(['poor']) and words_in_pheno_only == set()):
#         print "Opposite meaning due to word 'poor'", words_in_pheno_only, words_in_hpo_only
        return True
    for word in words_in_pheno_only:
        try:
            antonyms = get_antonyms(word)
        except UnicodeDecodeError as e:
            continue
        if antonyms.intersection(words_in_hpo_only):
#             print "Opposite meaning: ", antonyms.intersection(words_in_hpo_only)
            return True  
    for word in words_in_hpo_only:
        try:
            antonyms = get_antonyms(word)
        except UnicodeDecodeError as e:
            continue
        if antonyms.intersection(words_in_pheno_only):
#             print "Opposite meaning: ", antonyms.intersection(words_in_hpo_only)
            return True  
    return False 


# If the opposite meanings are found for a hpoid, then its synonyms are all in opposite meanings
unmatches_due_to_opposite_meanings = []

def map2hpo(pheno):
    global unmatches_due_to_opposite_meanings
    matches = []
    # Use lemmatizer to lemmatize words e.g., disturbances --> distrubance
    lemmatizer = WordNetLemmatizer()
    for hpo in hpo_terms:
        hpoid = hpo_name2id[hpo]
        pheno_ = pheno.lower() 
        hpo_ = hpo.lower()
        if hpo_.startswith('obsolete '):
            continue 
        if pheno_ == hpo_:
            matches.append((hpoid, hpo, 'EXACT', 1.0)) 
            continue
        pheno_words = [_.strip(",()'") for _ in re.split(' |/', pheno_)]
        hpo_words = [_.strip(",()'") for _ in re.split(' |/', hpo_)]

        '''
        # This is a corner case. Developmental delay means retarded and growth delay means short. 
        # But in plain language, they are exchangable.
        for pheno_word in pheno_words:
            if re.search('development', pheno_word, re.I):
                pheno_words.append('growth') 
            if re.search('growth', pheno_word, re.I):
                pheno_words.append('developmental') 
        '''
        try:
            pheno_words = [lemmatizer.lemmatize(word) for word in pheno_words]
            hpo_words = [lemmatizer.lemmatize(word) for word in hpo_words]
        except UnicodeDecodeError as e:
            pass
        if pheno_words == hpo_words:
            matches.append((hpoid, hpo, 'LEMMA_MATCH', 1.0)) 
            continue
        if set(pheno_words) == set(hpo_words):
            matches.append((hpoid, hpo, 'REORDER', 1.0)) 
            continue

        common_words = set(pheno_words) & set(hpo_words)
        words_in_pheno_only = set(pheno_words) - common_words
        words_in_hpo_only = set(hpo_words) - common_words
        # check if the pheno and hpo are in opposite meaning
        # if any word in pheno is an antonym of a word in hpo, or the other way
        # we skip this hpo
        sim = float(len(common_words)) / max(len(hpo_words), len(pheno_words))  
        if (sim >= 0.3333) and (pheno_ in hpo_) and (len(hpo_words) <= 10 * len(pheno_words)):
            if not isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
                matches.append((hpoid, hpo, 'SUB', sim))
            else:
                unmatches_due_to_opposite_meanings.append(hpoid.split('-')[0])
            continue
        if (sim >= 0.3333) and (hpo_ in pheno_) and (len(pheno_words) <= 10 * len(hpo_words)): 
            if not isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
                matches.append((hpoid, hpo, 'SUPER', sim))
            else:
                unmatches_due_to_opposite_meanings.append(hpoid.split('-')[0])
            continue
        if (sim >= 0.5) and len(pheno_words) >= 2:
            if not isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
                matches.append((hpoid, hpo, 'OVERLAP', sim))
            else:
                unmatches_due_to_opposite_meanings.append(hpoid.split('-')[0])
    matches_res = []
    for match in matches:
        if match[0].split('-')[0] not in unmatches_due_to_opposite_meanings:
            matches_res.append(match)
    matches = matches_res
    return matches

def filterMatchesOnCommonAncestors(matches):
    ## e.g., for "Central hypotonia", matches is like: 
    ## [('Central', 'SUPER', 0.5), ('Central hypotonia', 'EXACT', '1.0'), ('Hypotonia', 'SUPER', 0.5)]
    ## lca is like ('HP:0005872', 9, 'Brachytelomesophalangy')
    ## The function returns like: [('Stereotypical hand wringing', 'HP:0012171', 0.6666666666666666)]
    if not matches:
        return matches 
    final_matches = []
    # If the match is a disease not a phenotype, then we just add it to the final matches result
    matches_tmp = []
    for match in matches:
        if not match[0].startswith('HP:'):
            final_matches.append((match[1], match[0], match[3]))
        else:
            matches_tmp.append(match)
    matches = matches_tmp
    if not matches:
        return final_matches
    best_match_hpo = sorted(matches, key = lambda x: x[3], reverse = True)[0][0]
    for match in matches:
        term = match[0]
        # Remove 'synonym' from the hpoid (term) e.g., HP:0004414-synonym --> HP:0004414
        term = term.split('-')[0]
        lca, common_ancestors = findLowestCommonAncestor(term, best_match_hpo.split('-')[0], hpo_superclass)
        if lca[1] >= 2:
            final_matches.append((match[1], match[0], match[3])) 
    return final_matches    

def map2hpoWithPhenoSynonyms(pheno):
    global unmatches_due_to_opposite_meanings
    matches = map2hpo(pheno)
    direct_matches = filterMatchesOnCommonAncestors(matches)  
    final_matches = direct_matches 
    for match in direct_matches:
        hponame = match[0]
        hpoid = match[1].split('-')[0]
        sim = match[2]
        if sim >= 0.75:
            hposynonyms = hpo_id2synonyms[hpoid]
            for synonym in hposynonyms:
                if synonym != hponame:
#                     print synonym
                    matches_ = map2hpo(synonym)
                    indirect_matches = filterMatchesOnCommonAncestors(matches_)
                    final_matches = final_matches + indirect_matches
    final_matches = list(set(final_matches))
    matches_res = []
    for match in final_matches:
        if match[0].split('-')[0] not in unmatches_due_to_opposite_meanings:
            matches_res.append(match)
    final_matches = matches_res
    unmatches_due_to_opposite_meanings = []
    return final_matches

end = time.time()
print(end - start)

0.005784034729


In [95]:
start = time.time()

hpoid2gene = dict()
with open(PHENOTYPE_TO_GENE_FILE, 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        geneid, genesymbol, hponame, hpoid = parts[0], parts[1], parts[2], parts[3]
        if hpoid in hpoid2gene:
            hpoid2gene[hpoid].append(genesymbol)   
        else:
            hpoid2gene[hpoid] = [genesymbol]

end = time.time()
print(end - start)

3.22824883461


In [96]:
def map2gene(final_matches):
    # Map to genes for each phenotype (one phenotype at one time)
    # Pre-process the final_matches by remove '-synonym' from the hpoid and then unique the hpoids
    final_matches = [_[1] for _ in final_matches] 
    final_matches = [_.split('-')[0] for _ in final_matches] 
    final_matches = list(set(final_matches))
    mapped_genes = []
    for hpoid in final_matches:
        try:
            mapped_genes += hpoid2gene[hpoid] 
        except KeyError:
            pass
    ## For each phenotype in patient record, we generated multiple phenotype keywords 
    ## to maximize the mapping to the genes. However, we only count it once for each 
    ## phenotype even if the multiple keywords lead to multiple mappings.
    ## In practice, we don't need to screen through all genes. Only a few gene candidates are examined.
    mapped_genes = list(set(mapped_genes) & set(CANDIDATE_GENES))
    return mapped_genes

def map2geneWithSim(final_matches):
    # Map to genes for each phenotype (one phenotype at one time) with similarity information (word len)
    # final_matches like: [('Postnatal macrocephaly','HP:0005490',0.5), ('Macrocephaly,relative','HP:0004482-synonym', 0.5), ...]
    # map2gene function returns a list while this function returns a dict with genesymbol as key and similarity as value 
    hpoid_sim = dict() 
    for match in final_matches:
        hpoid = match[1].split('-')[0]
        sim = match[2]
        if hpoid not in hpoid_sim or sim > hpoid_sim[hpoid]:
            hpoid_sim[hpoid] = sim
    mapped_genes_score = dict()
    for hpoid in hpoid_sim:
        try:
            mapped_genes = hpoid2gene[hpoid]
        except KeyError:
            continue
        sim = hpoid_sim[hpoid]
        for gene in mapped_genes:
            # In practice, we don't need to screen through all genes. 
            # Only a few gene candidates are examined.
            if gene not in CANDIDATE_GENES:
                continue
            if gene not in mapped_genes_score or sim > mapped_genes_score[gene]:
                mapped_genes_score[gene] = sim
    return mapped_genes_score

In [97]:
start = time.time()


hpoid2disease = dict()
numphenosindisease = dict()
with open(PHENOTYPE_TO_DISEASE_FILE, 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        diseaseid, hpoid, hponame, diseasename = parts[0], parts[1], parts[2], parts[3]
        if diseasename in numphenosindisease:
            numphenosindisease[diseasename] += 1
        else:
            numphenosindisease[diseasename] = 1
        if hpoid in hpoid2disease:
            hpoid2disease[hpoid].append(diseasename)
        else:
            hpoid2disease[hpoid] = [diseasename]

end = time.time()
print(end - start)

0.259385824203


In [98]:
def map2disease(final_matches):
    # Map to diseases for each phenotype (one phenotype at one time)
    # Pre-process the final_matches by remove '-synonym' from the hpoid and then unique the hpoids
    final_matches = [_[1] for _ in final_matches] 
    final_matches = [_.split('-')[0] for _ in final_matches] 
    final_matches = list(set(final_matches))
    mapped_diseases = []
    for hpoid in final_matches:
        try:
            mapped_diseases += hpoid2disease[hpoid] 
        except KeyError:
            pass
    ## For each phenotype in patient record, we generated multiple phenotype keywords 
    ## to maximize the mapping to the genes. However, we only count it once for each 
    ## phenotype even if the multiple keywords lead to multiple mappings.
    ## In practice, we don't need to screen through all genes. Only a few gene candidates are examined.
    #mapped_genes = list(set(mapped_genes) & set(CANDIDATE_GENES))
    return mapped_diseases

def map2diseaseWithSim(final_matches):
    # Map to diseases for each phenotype (one phenotype at one time) with similarity information (word len)
    # final_matches like: [('Postnatal macrocephaly','HP:0005490',0.5), ('Macrocephaly,relative','HP:0004482-synonym', 0.5), ...]
    # map2disease function returns a list while this function returns a dict with diseasename as key and similarity as value 
    hpoid_sim = dict() 
    for match in final_matches:
        hpoid = match[1].split('-')[0]
        sim = match[2]
        if hpoid not in hpoid_sim or sim > hpoid_sim[hpoid]:
            hpoid_sim[hpoid] = sim
    mapped_diseases_score = dict()
    for hpoid in hpoid_sim:
        try:
            mapped_diseases = hpoid2disease[hpoid]
        except KeyError:
            continue
        sim = hpoid_sim[hpoid]
        for disease in mapped_diseases:
            # In practice, we don't need to screen through all genes. 
            # Only a few gene candidates are examined.
            #if disease not in CANDIDATE_DISEASES:
            #    continue
            if disease not in mapped_diseases_score or sim > mapped_diseases_score[disease]:
                mapped_diseases_score[disease] = sim
    return mapped_diseases_score

In [99]:
start = time.time()


all_mapped_genes = []
all_mapped_genes_score = {}
gene_phenos = {} 
all_mapped_diseases = []
all_mapped_diseases_score = {}
disease_phenos = {} 

for pheno in phenos:
#     print ("===========================================================================================")
#     pprint.pprint(pheno)
    matches =  map2hpo(pheno) 
    final_matches = map2hpoWithPhenoSynonyms(pheno)
#     pprint.pprint(final_matches)
    mapped_genes = map2gene(final_matches)
#     pprint.pprint(mapped_genes)
    all_mapped_genes += mapped_genes
    mapped_genes_score = map2geneWithSim(final_matches)
    if pheno in corner_cases:
        pheno = corner_cases[pheno]
    for gene in mapped_genes_score:
        if gene not in all_mapped_genes_score:
            all_mapped_genes_score[gene] = mapped_genes_score[gene]
        else:
            all_mapped_genes_score[gene] += mapped_genes_score[gene]
        if gene not in gene_phenos:
            gene_phenos[gene] = [pheno]
        else:
            gene_phenos[gene].append(pheno)

    mapped_diseases = map2disease(final_matches)
#     pprint.pprint(mapped_diseases)
    all_mapped_diseases += mapped_diseases
    mapped_diseases_score = map2diseaseWithSim(final_matches)
    for disease in mapped_diseases_score:
        if disease not in all_mapped_diseases_score:
            all_mapped_diseases_score[disease] = mapped_diseases_score[disease]
        else:
            all_mapped_diseases_score[disease] += mapped_diseases_score[disease]
        if disease not in disease_phenos:
            disease_phenos[disease] = [pheno]
        else:
            disease_phenos[disease].append(pheno)
            
end = time.time()
print(end - start)

260.649292946


In [100]:
start = time.time()


num_patient_phenos = len(phenos)
for disease in all_mapped_diseases_score:
    numphenos = numphenosindisease[disease]
    disease_score = all_mapped_diseases_score[disease]
#     print disease, numphenos, num_patient_phenos, disease_score
    # algo to calculate the disease score
    all_mapped_diseases_score[disease] = disease_score ** 2 / float(numphenos) / (1 + num_patient_phenos - disease_score)

all_mapped_genes_score = [(k,v) for k, v in all_mapped_genes_score.items()]   
all_mapped_genes_score = sorted(all_mapped_genes_score, key = lambda pair: pair[1], reverse = True)
all_mapped_diseases_score = [(k,v) for k, v in all_mapped_diseases_score.items()]   
all_mapped_diseases_score = sorted(all_mapped_diseases_score, key = lambda pair: pair[1], reverse = True)

count = Counter(all_mapped_genes)
count = sorted(count.items(), key = lambda pair: pair[1], reverse = True)
#print "********************************************************************************************"
#print "********************************************************************************************"
#print "Count of matches to phenos: "
#pprint.pprint(count[0:])
#print len(count)
#print "********************************************************************************************"
#print "********************************************************************************************"
#print "Score of matches to phenos: "
#pprint.pprint(all_mapped_genes_score[0:])
#print len(all_mapped_genes_score)
#print "********************************************************************************************"
#print "********************************************************************************************"
#print "Matched phenos: "
#pprint.pprint(gene_phenos)

#print "============================================================================================"
#print "============================================================================================"
count_disease = Counter(all_mapped_diseases)
count_disease = sorted(count_disease.items(), key = lambda pair: pair[1], reverse = True)
#print "********************************************************************************************"
#print "********************************************************************************************"
#print "Count of matches to phenos: "
#pprint.pprint(count_disease[0:])
#print "********************************************************************************************"
#print "********************************************************************************************"
#print "Score of matches to phenos: "
#pprint.pprint(all_mapped_diseases_score[0:])
#print "********************************************************************************************"
#print "********************************************************************************************"
#print "Matched phenos: "
#pprint.pprint(disease_phenos)

end = time.time()
print(end - start)

0.0956249237061


In [101]:
start = time.time()

with open(OUT_FILE, 'wb') as f:
    f.write('gene\tscore\thits\n')
    for id in xrange(len(all_mapped_genes_score)):
        gene, score = all_mapped_genes_score[id]
        hits = None
        for gene_hits in count:
            if gene_hits[0] == gene:
                hits = gene_hits[1] 
                break
        if hits: 
            f.write(gene + '\t' + str(score) + '\t' + str(hits) + '\n')

with open(OUT_FILE_DISEASE, 'wb') as f:
    f.write('disease\tscore\thits\n')
    for id in xrange(len(all_mapped_diseases_score)):
        disease, score = all_mapped_diseases_score[id]
        hits = None
        for disease_hits in count_disease:
            if disease_hits[0] == disease:
                hits = disease_hits[1] 
                break
        if hits:
            f.write(disease + '\t' + str(score) + '\t' + str(hits) + '\n')
            
end = time.time()
print(end - start)

0.879236936569


In [102]:
print(end - allstart)

267.042837858


In [103]:
len(phenos)

36

In [104]:
all_mapped_genes = []
all_mapped_genes_score = {}
gene_phenos = {} 
all_mapped_diseases = []
all_mapped_diseases_score = {}
disease_phenos = {} 

for pheno in phenos:

    start = time.time()
    matches = map2hpo(pheno) 
    end = time.time()
    print 'map2hpo', (end - start)
    
    start = time.time()
    final_matches = map2hpoWithPhenoSynonyms(pheno)
    end = time.time()
    print 'map2hpoWithPhenoSynonyms', (end - start)
    
    start = time.time()
    mapped_genes = map2gene(final_matches)
    end = time.time()
    print 'map2gene', (end - start)
    
    all_mapped_genes += mapped_genes
    
    start = time.time()
    mapped_genes_score = map2geneWithSim(final_matches)
    end = time.time()
    print 'map2geneWithSim', (end - start)
    
    start = time.time()
    if pheno in corner_cases:
        pheno = corner_cases[pheno]
    for gene in mapped_genes_score:
        if gene not in all_mapped_genes_score:
            all_mapped_genes_score[gene] = mapped_genes_score[gene]
        else:
            all_mapped_genes_score[gene] += mapped_genes_score[gene]
        if gene not in gene_phenos:
            gene_phenos[gene] = [pheno]
        else:
            gene_phenos[gene].append(pheno)
    end = time.time()
    print 'mapped_genes_score', (end - start)
    
    start = time.time()
    mapped_diseases = map2disease(final_matches)
    end = time.time()
    print 'map2disease', (end - start)
    
    all_mapped_diseases += mapped_diseases
    
    start = time.time()
    mapped_diseases_score = map2diseaseWithSim(final_matches)
    end = time.time()
    print 'map2diseaseWithSim', (end - start)
    
    start = time.time()
    for disease in mapped_diseases_score:
        if disease not in all_mapped_diseases_score:
            all_mapped_diseases_score[disease] = mapped_diseases_score[disease]
        else:
            all_mapped_diseases_score[disease] += mapped_diseases_score[disease]
        if disease not in disease_phenos:
            disease_phenos[disease] = [pheno]
        else:
            disease_phenos[disease].append(pheno)
    end = time.time()
    print 'all_mapped_diseases_score', (end - start)
    
    break

map2hpo 2.3309879303
map2hpoWithPhenoSynonyms 2.36133313179
map2gene 4.6968460083e-05
map2geneWithSim 5.96046447754e-06
mapped_genes_score 9.53674316406e-07
map2disease 3.91006469727e-05
map2diseaseWithSim 1.50203704834e-05
all_mapped_diseases_score 9.53674316406e-07


In [3]:
start = time.time()

import map_phenotype_to_gene
import pandas as pd
import re

input_phenotype = 'data/sample_patient_phenotype.txt'
input_genes = 'data/sample_genes.txt'

df_genes = pd.read_csv(input_genes, sep = '\t')
CANDIDATE_GENES = pd.unique(df_genes.Gene.values).tolist()

corner_cases = dict()
with open(input_phenotype, 'rb') as infile:
    phenos = []
    for line in infile:
        if line.startswith('#') or not line.strip():
            continue
        line = line.rstrip()
        phenos += line.split(',')
        for pheno in phenos:
            if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
                phenos.append('growth delay')
                corner_cases['growth delay'] = pheno.strip()
        for pheno in phenos:
            if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
                phenos.append('developmental delay')
                corner_cases['developmental delay'] = pheno.strip()
    phenos = [_.strip() for _ in phenos]

ranking_genes, ranking_disease = map_phenotype_to_gene.generate_score(phenos, CANDIDATE_GENES, corner_cases)

end = time.time()
print end-start

165.858657837


In [16]:
len(ranking_disease)

3322

In [1]:
# ranking_genes

In [2]:
# np.unique([i[0] for i in ranking_genes]).tolist()

In [1]:
import time

In [4]:
start = time.time()

import map_phenotype_to_gene
import collectVariantInfo

import pandas as pd
import numpy as np
import re
import myvariant


input_phenotype = 'data/sample_patient_phenotype.txt'
input_genes = 'data/sample_genes.txt'

df_genes = pd.read_csv(input_genes, sep = '\t', usecols = [0])
CANDIDATE_GENES = pd.unique(df_genes.Gene.values).tolist()

corner_cases = dict()
with open(input_phenotype, 'rb') as infile:
    phenos = []
    for line in infile:
        if line.startswith('#') or not line.strip():
            continue
        line = line.rstrip()
        phenos += line.split(',')
        for pheno in phenos:
            if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
                phenos.append('growth delay')
                corner_cases['growth delay'] = pheno.strip()
        for pheno in phenos:
            if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
                phenos.append('developmental delay')
                corner_cases['developmental delay'] = pheno.strip()
    phenos = [_.strip() for _ in phenos]

mv = myvariant.MyVariantInfo()

ranking_genes, ranking_disease = map_phenotype_to_gene.generate_score(phenos, CANDIDATE_GENES, corner_cases)

hpo_filtered_genes = np.unique([i[0] for i in ranking_genes]).tolist()

candidate_vars = []
with open(input_genes, 'rb') as f:
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        gene = parts[0]
        if gene not in hpo_filtered_genes:
            continue
        for part in parts:
            if re.search(r'_.*:', part):
                transcript, variant = part.split(':') 
        candidate_vars.append((gene, variant, transcript))

df_final_res1 = collectVariantInfo.get_variants(candidate_vars)

end = time.time()

In [3]:
t0 = time.time()


import map_phenotype_to_gene
import collectVariantInfo
import pubmed

import pandas as pd
import numpy as np
import re
import myvariant

#input 
input_phenotype = 'data/sample_patient_phenotype.txt'
input_genes = 'data/sample_genes.txt'

df_genes = pd.read_csv(input_genes, sep = '\t', usecols = [0])
CANDIDATE_GENES = pd.unique(df_genes.Gene.values).tolist()

corner_cases = dict()
with open(input_phenotype, 'rb') as infile:
    phenos = []
    for line in infile:
        if line.startswith('#') or not line.strip():
            continue
        line = line.rstrip()
        phenos += line.split(',')
        for pheno in phenos:
            if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
                phenos.append('growth delay')
                corner_cases['growth delay'] = pheno.strip()
        for pheno in phenos:
            if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
                phenos.append('developmental delay')
                corner_cases['developmental delay'] = pheno.strip()
    phenos = [_.strip() for _ in phenos]


# map phenotype to gene
mv = myvariant.MyVariantInfo()

ranking_genes, ranking_disease = map_phenotype_to_gene.generate_score(phenos, CANDIDATE_GENES, corner_cases)

t1 = time.time()

# collect variant info
hpo_filtered_genes = np.unique([i[0] for i in ranking_genes]).tolist()

candidate_vars = []
with open(input_genes, 'rb') as f:
	f.readline()
	for line in f.readlines():
		line = line.rstrip()
		parts = line.split('\t')
		gene = parts[0]
		if gene not in hpo_filtered_genes:
			continue
		for part in parts:
			if re.search(r'_.*:', part):
		  		transcript, variant = part.split(':') 
		candidate_vars.append((gene, variant, transcript))

final_res = collectVariantInfo.get_variants(candidate_vars)

t2 = time.time()

# pubmed
pubmed_query_results = pubmed.queryPubmedDB(final_res)



t3 = time.time()

In [11]:
t3-t0

78.045254945755

In [21]:
pd.DataFrame(ranking_genes, columns=['gene','score','hits'])

,gene,score,hits
0,FLNA,16.333333,21
1,LMNA,14.833333,21
2,FANCL,14.000000,17
3,GNAS,14.000000,20
4,FANCM,13.000000,16
5,MEGF8,10.666667,15
6,CAMKMT,9.500000,11
7,WWOX,9.500000,13
8,PRKCSH,9.333333,13
9,PLEC,9.000000,13


In [4]:
import time

In [5]:
t1 = time.time()

input_phenotype = 'data/sample_patient_phenotype.txt'
input_genes = 'data/sample_genes.txt'

import main
acmg_result= main.master_function(input_phenotype,input_genes)

t2 = time.time()

In [6]:
t2-t1

164.84264588356018

In [7]:
acmg_result

,gene,variant,id,final_score,pathogenicity_score,pathogenicity,hit_criteria,hpo_hit_score
0,CBS,c.833T>C,chr21:g.44483184A>G,7.783641,4.000000,Likely pathogenic,PS1|PM5|PP3|PP5,6.000000
1,EXOSC8,c.815G>C,chr13:g.37583420G>C,7.708397,5.125000,Pathogenic,PS1|PM5|PM1|PP3|PP2|PP5,3.500000
2,MEGF8,c.8180G>A,chr19:g.42880569G>A,5.527655,2.250000,Uncertain significance,PM1|PM2|PP3,10.666667
3,CLCNKA,c.910C>T,chr1:g.16354556C>T,5.132180,3.500000,Uncertain significance,PVS1|PP3,3.333333
4,PGAP3,c.67G>A,chr17:g.37844201C>T,5.083242,2.333333,Uncertain significance,PM1|PM2|PP3,7.833333
5,CCBE1,c.344C>T,chr18:g.57136761G>A,4.154201,1.875000,Uncertain significance,PM1|PP3|PP2,8.166667
6,GNAS,c.*293+1G>A,chr20:g.57478847G>A,4.062075,1.500000,Uncertain significance,PM2|PP3,14.000000
7,FLNA,c.3611C>T,chrX:g.153588552G>A,3.922368,1.375000,Uncertain significance,PM2|PP3,16.333333
8,RTTN,c.4275G>C,chr18:g.67755252C>G,3.420056,2.125000,Uncertain significance,PM1|PM2|PP3,4.000000
9,WNT7A,c.299-4G>A,chr3:g.13896304C>T,3.376938,1.500000,Uncertain significance,PM2|PP3,8.500000
